# BOOTCAMP AI GS

## Case 2

Seu time faz parte de uma start-up de grande sucesso no setor de aluguel
de casas para curtas durações. A plataforma vem experimentando um crescimento
significativo, com o site e o aplicativo atraindo cada vez mais usuários.

Atualmente, o mercado principal da empresa é a Europa, onde a oferta de
acomodações continua aumentando. Com esse crescimento acelerado, a demanda
por decisões baseadas em dados tem se intensificado, tanto para análises preditivas
quanto prescritivas.
Neste momento de expansão, foi criada uma equipe, a de Data Product
Management.

### Objetivo
 O primeiro objetivo dessa equipe é entregar um produto de dados: um
modelo de regressão capaz de prever o valor esperado do aluguel com base em
informações sobre a acomodação (como número máximo de pessoas, quantidade de
quartos, distância ao centro da cidade, latitude, longitude, dia da semana, entre outras
features).


### Análise Exploratória

Importando as bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score


Leitura das tabelas

In [ ]:
df_barcelona_weekdays =  pd.read_csv('data/barcelona_weekdays.csv')
df_barcelona_weekends=  pd.read_csv('data/barcelona_weekends.csv')

df_berlin_weekdays =  pd.read_csv('data/berlin_weekdays.csv')
df_berlin_weekends =  pd.read_csv('data/berlin_weekends.csv')

df_paris_weekdays =  pd.read_csv('data/barcelona_weekdays.csv')
df_paris_weekends =  pd.read_csv('data/barcelona_weekends.csv')

Atribuindo colunas indicativas

In [ ]:
# Incluindo o tipo de dia

df_barcelona_weekdays["type"] = "weekdays"
df_berlin_weekdays["type"] = "weekdays"
df_paris_weekdays["type"] = "weekdays"

df_barcelona_weekends["type"] = "weekends"
df_berlin_weekends["type"] = "weekends"
df_paris_weekends["type"] = "weekends"

# Incluindo o nome da cidade

df_barcelona_weekdays["city"] = "barcelona"
df_barcelona_weekends["city"] = "barcelona"

df_berlin_weekdays["city"] = "berlin"
df_berlin_weekends["city"] = "berlin"

df_paris_weekdays["city"] = "paris"
df_paris_weekends["city"] = "paris"



Concatenando para um só dataset

In [ ]:
df_original = pd.concat([df_barcelona_weekdays, df_berlin_weekdays, df_paris_weekdays, df_barcelona_weekends, df_berlin_weekends, df_paris_weekends])

In [ ]:
df_original.drop(df_original.columns[[0, 14, 16]], axis=1, inplace=True) 

In [ ]:
df_original.head()

In [ ]:
df_original.shape

Tipos de Dados

In [ ]:
df_original.dtypes

Valores Nulos

In [ ]:
df_original.isna().sum()

In [ ]:
df_original.nunique()

In [ ]:
df = df_original.copy()

continuous = []
categorical = []

for c in df.columns[:-2]:
    if df.nunique()[c]>=30:
        continuous.append(c)
    else:
        categorical.append(c)

In [ ]:
continuous 

In [ ]:
df[categorical].head()

In [ ]:
fig = plt.figure(figsize=(12,8))

for i, col in enumerate(continuous):
    plt.subplot(3,3,i+1)
    df.boxplot(col)
    plt.tight_layout()

## Transformação logarítmica nos dados contínuos

Reduz o impacto de outliers <br>
Torna a distribuição mais normal <br>
Facilita visualização e modelagem estatística 

In [ ]:
df[continuous] = np.log1p(1 + df[continuous])

In [ ]:
fig = plt.figure(figsize=(12,8))

for i, col in enumerate(continuous):
    plt.subplot(3,3,i+1)
    df.boxplot(col)
    plt.tight_layout()

### Mapa de correlação variáveis contínuas

A correlação varia entre -1 e 1: <br>
 0 - Nenhuma correlação (as variáveis não têm relação linear). <br>
+1 - Correlação positiva (se A aumenta, B aumenta proporcionalmente). <br>
-1 - Correlação negativa (se A aumenta, B diminui proporcionalmente). <br>

### Escala comum para interpretação

0.0 – 0.2 → Correlação muito fraca ou insignificante <br>
0.2 – 0.4 → Correlação fraca <br>
0.4 – 0.6 → Correlação moderada <br>
0.6 – 0.8 → Correlação forte <br>
0.8 – 1.0 → Correlação muito forte <br>

In [ ]:
plt.figure(figsize=(8,8))

sns.heatmap(df[['realSum',
                'guest_satisfaction_overall',
                'dist',
                'metro_dist',
                'attr_index_norm',
                'rest_index_norm',
                'lng',
                'lat']].corr(), vmax = 1, square = True, cmap='coolwarm', annot=True  )


In [ ]:
plt.figure(figsize=(10,12))

sns.scatterplot(data=df, x='dist', y='realSum')
plt.xlabel('Distância do centro (km)')
plt.yticks(np.log1p([50, 100, 200, 500, 1000, 2000, 5000]), labels=[50, 100, 200, 500, 1000, 2000, 5000])
plt.ylabel('Log(Preço)')
plt.title('Relação entre Distância e Preço')

In [ ]:
plt.figure(figsize=(8,8))

sns.boxplot(x=df['realSum'])
plt.xticks(np.log1p([50, 100, 500, 1000, 5000]), labels=[50, 100, 500, 1000, 5000])

plt.title('Distribuição dos preços')

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(data=df, x="city", y="realSum") 
plt.yticks(np.log1p([50, 100, 500, 1000, 5000]), labels=[50, 100, 500, 1000, 5000])
plt.title("Distribuição de Preços por Cidade")


In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(data=df, x="type", y="realSum")
plt.xticks([0, 1], ["Dia útil", "Fim de semana"])
plt.title("Preços em Dias Úteis vs. Fim de Semana")
plt.xlabel("Período")
plt.ylabel("Preço (€)")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.barplot(data=df, x="city", y="cleanliness_rating")
plt.title("Taxa de Limpeza por Cidade")


In [ ]:
plt.xlabel("Tipos de Local")
sns.countplot(x=df['room_type'])
plt.show()

In [ ]:
labels = ['Quarto Privado', 'Casa/Ap Completo', 'Quarto Compartilhado']
plt.title("Tipos de Local")
plt.pie(df['room_type'].value_counts(), labels = labels, autopct="%.2f%%")
plt.legend()

In [ ]:
plt.figure(figsize=(8, 6))
df.groupby("city")["realSum"].mean().plot(kind="bar", color=["blue", "red", "green"])
plt.title("Média de Preço por Cidade")
plt.xlabel("Cidade")
plt.ylabel("Preço Médio (€)")
plt.show()

# Pré-Processamento dos Dados

In [ ]:
df_original['room_shared'] = df_original['room_shared'].astype(int)
df_original['room_private'] = df_original['room_private'].astype(int)
df_original['host_is_superhost'] = df_original['host_is_superhost'].astype(int)
df_original['room_type'] = df_original['room_type'].astype('category')
df_original['type'] = df_original['type'].astype('category')
df_original['city'] = df_original['city'].astype('category')


In [ ]:
df_original.dtypes

Removendo outliers

In [ ]:
Q1 = df_original['realSum'].quantile(0.25)
Q3 = df_original['realSum'].quantile(0.75)
IQR = Q3 - Q1

df_original = df_original[(df_original['realSum'] >= Q1 - 1.5 * IQR) & (df_original['realSum'] <= Q3 + 1.5 * IQR)]

One-Hot Encoding

In [ ]:
df_original  = pd.get_dummies(df_original, columns=['room_type', 'type', 'city'], drop_first=True)

In [ ]:
y = df_original['realSum']
X = df_original.drop(columns=['realSum'])

Divisão treino e teste

In [ ]:
X_train, X_teste, y_train, y_teste = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Criando o Modelo

### Regressão Linear

In [ ]:
modelo = LinearRegression()
modelo.fit(X_train, y_train)

In [ ]:
y_pred = modelo.predict(X_teste)

mae = mean_absolute_error(y_teste, y_pred)
r2 = r2_score(y_teste, y_pred)

print(f'MAE: {mae:.2f}')
print(f'R²: {r2:.2f}')

### Árvore de Regressão

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_teste)

mae_rf = mean_absolute_error(y_teste, y_pred_rf)
r2_rf = r2_score(y_teste, y_pred_rf)

print(f"Random Forest - MAE: {mae_rf:.2f}")
print(f"Random Forest - R²: {r2_rf:.2f}")


### Ridge

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, r2_score

ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
pred_ridge = ridge.predict(X_teste)

mae_ridge = mean_absolute_error(y_teste, pred_ridge)
r2_ridge =  r2_score(y_teste, pred_ridge)

print("Ridge")
print(f"MAE: {mae_ridge:.2f}")
print(f"R²: {r2_ridge:.2f}")


### ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, r2_score

elastic_model = make_pipeline(
    StandardScaler(),
    ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42)
)

elastic_model.fit(X_train, y_train)

y_pred_elastic = elastic_model.predict(X_teste)

mae_elastic = mean_absolute_error(y_teste, y_pred_elastic)
r2_elastic = r2_score(y_teste, y_pred_elastic)

print(f"ElasticNet - MAE: {mae_elastic:.2f}")
print(f"ElasticNet - R²: {r2_elastic:.2f}")


### XGBoost

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score

xgb_model = XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1,
reg_alpha=0.5, reg_lambda=1.0, random_state=42)

xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_teste)

mae_xgb = mean_absolute_error(y_teste, y_pred_xgb)
r2_xgb = r2_score(y_teste, y_pred_xgb)

print(f"XGBoost - MAE: {mae_xgb:.2f}")
print(f"XGBoost - R²: {r2_xgb:.2f}")
